In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
base_url = 'https://www.torrentjogos.com.br'

In [3]:
def get_game_details(url):
    # Configuração do driver do Chrome
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Caso queira rodar em modo headless
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    
    # Inicialização do WebDriver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

    url_completed = base_url + url
    # Acessa a página
    

    try:
        driver.get(url_completed)
        # Espera o elemento de descrição estar presente
        game_desc_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".PageGame_desc__AjwkG"))
        )

        # Pegando a sinopse do jogo
        game_description = game_desc_element.text

        game_minimum_requiriments_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".PageGame_minimum__D2ve3"))
        )

        # Pegando a sinopse do jogo
        game_minimum_requiriments = game_minimum_requiriments_element.text


        game_req_requiriments_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".PageGame_minimum__D2ve3"))
        )

        # Pegando a sinopse do jogo
        game_req_requiriments = game_req_requiriments_element.text
        

        # Espera o elemento de categorias estar presente
        genre_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".PageGame_genreItem__JyyVV"))
        )

        # Pegando as categorias
        categories = [genre.text for genre in genre_elements]

        # Pegando o magnetic link (torrent)
        magnet_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".PageGame_infodownloadinfo__Ft_UD a"))
        )

        magnet_link = magnet_element.get_attribute('href')

        iframe_element = driver.find_element(By.CLASS_NAME, "PageGame_videoFrame__XLjMv")

        # Alternar para o iframe
        driver.switch_to.frame(iframe_element)

        # Agora que estamos dentro do iframe, podemos capturar o atributo 'src'
        trailer_src = driver.current_url  # Isso vai retornar o URL do conteúdo dentro do iframe

        # Volte para o contexto principal (fora do iframe)
        driver.switch_to.default_content()

        # Espera até que a imagem esteja presente na página
        image_elements = driver.find_elements(By.CLASS_NAME, 'PageGame_modal_image__epuHK')

        images_src = [image.get_attribute('src') for image in image_elements]

        div_elements = driver.find_elements(By.CLASS_NAME, 'PageGame_iframeSteam__NgF1A')

        # Verifica se a lista de elementos encontrados não está vazia
        if div_elements:
            # Dentro da primeira div encontrada, localiza o iframe e pega o atributo 'src'
            iframe_shop_src_element = div_elements[0].find_element(By.TAG_NAME, 'iframe')
            shop_src = iframe_shop_src_element.get_attribute('src')
        else:
            # Caso a classe não seja encontrada, define um valor padrão para 'shop_src'
            shop_src = None 

        # Retorna os dados em um dicionário
        game_details = {
            "link": url,
            "shop_src":shop_src,
            "images_src":images_src,
            "description": game_description,
            "categories": categories,
            "magnet_link": magnet_link,
            "trailer_src": trailer_src,
            "game_minimum_requiriments":game_minimum_requiriments,
            "game_req_requiriments":game_req_requiriments,
        }

        return game_details

    except Exception as e:
        print(f"no link: {url}")
        print(f"Ocorreu um erro: {e}")
        return None
    finally:
        # Fecha o navegador
        driver.quit()

# Executando a função
# game_details = get_game_details('/World-War-Z-Aftermath')
# if game_details:
#     print(game_details)

In [4]:
cards = pd.read_excel('dados-jogos.xlsx')
cards.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      18 non-null     object
 1   link       18 non-null     object
 2   date       18 non-null     object
 3   size       18 non-null     object
 4   source     18 non-null     object
 5   image_url  18 non-null     object
dtypes: object(6)
memory usage: 992.0+ bytes


In [5]:
cards_details = []
cards_aux = cards
for idx,link in enumerate(cards_aux['link'],1):
    cards_details.append(get_game_details(link))
    print(idx)

cards_details = [item for item in cards_details if item is not None]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [6]:
#df_cards_details_old = pd.read_excel('dados-jogos-details.xlsx')
df_cards_details = pd.DataFrame(cards_details)
df_combined = df_cards_details

In [7]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   link                       18 non-null     object
 1   shop_src                   16 non-null     object
 2   images_src                 18 non-null     object
 3   description                18 non-null     object
 4   categories                 18 non-null     object
 5   magnet_link                18 non-null     object
 6   trailer_src                18 non-null     object
 7   game_minimum_requiriments  18 non-null     object
 8   game_req_requiriments      18 non-null     object
dtypes: object(9)
memory usage: 1.4+ KB


In [8]:
df_cards_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   link                       18 non-null     object
 1   shop_src                   16 non-null     object
 2   images_src                 18 non-null     object
 3   description                18 non-null     object
 4   categories                 18 non-null     object
 5   magnet_link                18 non-null     object
 6   trailer_src                18 non-null     object
 7   game_minimum_requiriments  18 non-null     object
 8   game_req_requiriments      18 non-null     object
dtypes: object(9)
memory usage: 1.4+ KB


In [9]:
df_combined.to_excel('dados-jogos-details.xlsx', index=False)